In [1]:
import pandas as pd
import numpy as np
import sys 
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
import itertools
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier


from sklearn.preprocessing import StandardScaler, MinMaxScaler
from joblib import Parallel, delayed, dump, load
# sys.path.insert(0, '../DevCode')


pd.set_option('display.expand_frame_repr', False)
pd.options.display.max_rows = 500
sys.path.append('../code')
import pickle

In [2]:
scaler = MinMaxScaler(feature_range=(0,1))

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

In [4]:
# valid = pd.read_pickle('../input/valid_tuple.pkl')
# train = pd.read_pickle('../input/train_tuple.pkl')

In [5]:
# valid = pd.read_pickle('../code/data/valid_tuple.pkl')
# train = pd.read_pickle('../code/data/train_tuple.pkl')
# test = pd.read_pickle('../code/data/test_tuple.pkl')

In [4]:
valid = pd.read_pickle('../code/data/scaled_valid_tuple.pkl')
train = pd.read_pickle('../code/data/scaled_train_tuple.pkl')
test = pd.read_pickle('../code/data/scaled_test_tuple.pkl')

In [5]:
X_train, y_train = train[0], train[1]
X_val, y_val = valid[0], valid[1]
X_test, y_test = test[0], test[1]

In [6]:
X_train = X_train[:, ::-1].copy()
X_val = X_val[:, ::-1].copy()
X_test = X_test[:, ::-1].copy()

In [11]:
X_train[0]

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 7.98896883e-06, 1.11586505e-05, 7.97200233e-06,
       9.56451184e-06, 7.98264254e-06, 6.37624974e-06, 3.20083478e-06,
       4.78255325e-06, 1.54816736e-05, 0.00000000e+00])

In [123]:
X_test[0]

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 8.18571932e-01, 1.11589173e-05,
       1.08587879e-02, 6.37908680e-06, 4.53032544e-03, 6.37911732e-06,
       2.22694427e-03, 6.39590662e-06, 4.86127181e-03, 1.11801798e-05,
       1.11587394e-05, 2.88078818e-05, 2.55119938e-05])

In [124]:
X_train[-1]

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 4.31240327e-05, 4.56646339e-02, 4.64222084e-05,
       5.75314241e-01, 3.18989950e-06, 3.81340045e-01, 3.18955357e-06,
       5.75079704e-01, 3.19214987e-06, 5.75075360e-01, 3.18826209e-06,
       5.75071016e-01, 4.30588359e-05, 5.10099980e-05, 3.98694833e-05,
       1.59409039e-05, 6.39590662e-06, 6.36465922e-02, 1.11801798e-05,
       7.97052818e-06, 6.40175152e-06, 5.50102367e-04])

In [ ]:
embedding = nn.Embedding(
    num_embeddings=10,
    embedding_dim=3
)

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, layers,output_size=4):
        super(LSTM, self).__init__()

        self.hidden_size = hidden_size

        self.layers = layers

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=layers, batch_first=True)

        # self.dropout = nn.Dropout(p=0.1)
        # self.fc = nn.Linear(hidden_size, output_size)


        self.fc = nn.Linear(hidden_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        

    def forward(self, x):

        h0 = torch.zeros(self.layers, x.size(0), self.hidden_size).requires_grad_().to(device)
        c0 = torch.zeros(self.layers, x.size(0), self.hidden_size).requires_grad_().to(device)

        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        
        # out = self.dropout(out)

        out = self.fc(out[:, -1, :])
        out = self.relu(out)
        out = self.fc2(out)
        # out = self.fc(out[-1, :, :])
        
        return out

In [11]:
class LSTMAttentionClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMAttentionClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.attention = nn.Linear(hidden_size, 1)
        self.fc = nn.Linear(hidden_size, num_classes)  # Adapted for classification
        
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        attn_weights = torch.softmax(self.attention(lstm_out), dim=1)
        context = torch.sum(attn_weights * lstm_out, dim=1)
        out = self.fc(context)
        return out

In [12]:
# class StackedLSTM(nn.Module):
#     def __init__(self, input_size, hidden_size, num_layers, output_size, dropout_rate):
#         super(StackedLSTM, self).__init__()
#         self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout_rate)
#         self.fc = nn.Linear(hidden_size, output_size)
        
#     def forward(self, x):
#         out, _ = self.lstm(x)
#         out = self.fc(out[:, -1, :])
#         return out
    
class StackedLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout_rate):
        super(StackedLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        # self.fc = nn.Linear(hidden_size, output_size)
        self.fc = nn.Linear(hidden_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out

In [78]:
class BiLSTMClassifierNoEmbedding(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(BiLSTMClassifierNoEmbedding, self).__init__()
        # No embedding layer here
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)  # We multiply by 2 for bidirectional

        self.fc2 = nn.Linear(hidden_dim * 2, hidden_dim)

        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        # Assuming x is already in the correct shape: [batch_size, seq_len, input_dim]
        lstm_out, (hidden, cell) = self.lstm(x)
        # Concatenate the final forward and backward hidden states
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)

        out = self.fc2(hidden)
        out = self.relu(out)
        out = self.fc3(out)
        return out


        #dense_outputs = self.fc(hidden)
        #return dense_outputs


In [79]:
features = X_train
labels = y_train

In [10]:
labels.shape

(440619,)

In [11]:
labels

array([1, 0, 0, ..., 0, 0, 0])

In [185]:
features.shape

(440619, 47)

In [13]:
# features = scaler.fit_transform(features)

In [80]:
features = features.reshape(-1, 47,1)
labels = labels.reshape(-1)

In [155]:
labels.shape

(440619,)

In [158]:
features[0]

array([[0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],


In [133]:
features.shape

(440619, 47, 1)

In [81]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from torch.optim.lr_scheduler import StepLR
import numpy as np

# Example data
# features = np.random.randint(0, 100, (440619, 47))
# labels = np.random.randint(0, 2, (440619,))

# Convert to tensors
features_tensor = torch.tensor(features, dtype=torch.float32)
labels_tensor = torch.tensor(labels, dtype=torch.long)

In [82]:
'1024'

# Create dataset and dataloader
dataset = TensorDataset(features_tensor, labels_tensor)
data_loader = DataLoader(dataset, batch_size=1024, shuffle=True)


In [77]:
try:
    del model, optimizer, loss, criterion
except:
    pass

In [144]:
# Model parameters
# input_dim = 47

# hidden_dim = 128
# hidden_dim = 42

hidden_dim = 46

input_dim = 1
hidden_dim = 256
output_dim = 4  # Assuming binary classification
num_layers = 3
# num_layers = 2

learing_rate = 0.001


    

# model = LSTM(input_size=input_dim, hidden_size=hidden_dim, layers=num_layers).to(device)

# model = LSTMAttentionClassifier(input_size=input_dim, hidden_size=hidden_dim, num_layers=num_layers, num_classes=output_dim).to(device)

model = StackedLSTM(input_size=input_dim, hidden_size=hidden_dim, num_layers=num_layers, output_size=output_dim, dropout_rate=0.01).to(device)

#model = BaseNN(input_size=47, hidden_size=256, output_size=output_dim).to(device)

# Initialize model, loss function, and optimizer
# model = LSTMClassifier(input_size=input_dim, hidden_size=hidden_dim, layers=num_layers,output_dim, num_layers).to(device)
criterion = nn.CrossEntropyLoss()
# optimizer = optim.AdamW(model.parameters(), lr=0.001)
# optimizer = optim.AdamW(model.parameters(), lr=0.0005)
optimizer = optim.Adam(model.parameters(), lr=0.00025)

In [85]:

# hidden_dim = 46

input_dim = 1
hidden_dim = 128
output_dim = 4  # Assuming binary classification
num_layers = 3
# num_layers = 2

learing_rate = 0.001


model = BiLSTMClassifierNoEmbedding(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim).to(device)

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=0.02)

scheduler = StepLR(optimizer, step_size=25, gamma=0.5)

In [86]:
#lr=0.01

In [87]:
print(model)
print(len(list(model.parameters())))
for i in range(len(list(model.parameters()))):
    print(list(model.parameters())[i].size())

BiLSTMClassifierNoEmbedding(
  (lstm): LSTM(1, 128, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=256, out_features=4, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (relu): ReLU()
  (fc3): Linear(in_features=128, out_features=4, bias=True)
)
14
torch.Size([512, 1])
torch.Size([512, 128])
torch.Size([512])
torch.Size([512])
torch.Size([512, 1])
torch.Size([512, 128])
torch.Size([512])
torch.Size([512])
torch.Size([4, 256])
torch.Size([4])
torch.Size([128, 256])
torch.Size([128])
torch.Size([4, 128])
torch.Size([4])


In [67]:
# model.load_state_dict(torch.load('../code/data/model.pt'))
# optimizer.load_state_dict(torch.load('../code/data/optimizer.pt'))

In [68]:
# optimizer.param_groups[0]['lr'] = 0.0001
# optimizer.param_groups[0]['lr'] = 0.00005

In [69]:
# torch.save(model.state_dict(), '../code/data/model.pt')
# torch.save(optimizer.state_dict(), '../code/data/optimizer.pt')

In [70]:
crossed = False

In [88]:


# Training loop
# num_epochs = 150
num_epochs = 300
for epoch in range(num_epochs):
    for inputs, labels in data_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        # Forward pass

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    scheduler.step()



        # inputs = inputs.cpu()
        # labels = labels.cpu()
        

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')



Epoch [1/300], Loss: 0.3406
Epoch [2/300], Loss: 0.2706
Epoch [3/300], Loss: 0.1776
Epoch [4/300], Loss: 0.2319
Epoch [5/300], Loss: 0.2015
Epoch [6/300], Loss: 0.1982
Epoch [7/300], Loss: 0.2195
Epoch [8/300], Loss: 0.2214
Epoch [9/300], Loss: 0.1749
Epoch [10/300], Loss: 0.1831
Epoch [11/300], Loss: 0.1419
Epoch [12/300], Loss: 0.1655
Epoch [13/300], Loss: 0.1028
Epoch [14/300], Loss: 0.1022
Epoch [15/300], Loss: 0.1594
Epoch [16/300], Loss: 0.1841
Epoch [17/300], Loss: 0.1233
Epoch [18/300], Loss: 0.1786
Epoch [19/300], Loss: 0.0868
Epoch [20/300], Loss: 0.1280
Epoch [21/300], Loss: 0.1442
Epoch [22/300], Loss: 0.1592
Epoch [23/300], Loss: 0.3096
Epoch [24/300], Loss: 0.1196
Epoch [25/300], Loss: 0.1496
Epoch [26/300], Loss: 0.1864
Epoch [27/300], Loss: 0.1379
Epoch [28/300], Loss: 0.1200
Epoch [29/300], Loss: 0.1124
Epoch [30/300], Loss: 0.1271
Epoch [31/300], Loss: 0.0917
Epoch [32/300], Loss: 0.1446
Epoch [33/300], Loss: 0.1098
Epoch [34/300], Loss: 0.1347
Epoch [35/300], Loss: 0

In [19]:
del model, optimizer

In [72]:
torch.save(model.state_dict(), '../code/data/model2.pt')
torch.save(optimizer.state_dict(), '../code/data/optimizer2.pt')

In [49]:
torch.save(model, '../code/data/model_main.pt')
# torch.save(optimizer.state_dict(), '../code/data/optimizer.pt')

In [36]:
valid_features = X_test
valid_labels = y_test

valid_features = scaler.fit_transform(valid_features)

valid_features = valid_features.reshape(-1, 47,1)
valid_labels = valid_labels.reshape(-1)

valid_features_tensor = torch.tensor(valid_features, dtype=torch.float32)
valid_labels_tensor = torch.tensor(valid_labels, dtype=torch.long)


valid_dataset = TensorDataset(valid_features_tensor, valid_labels_tensor)
valid_data_loader = DataLoader(valid_dataset, batch_size=512, shuffle=True)

In [73]:
val_bool = False

if val_bool:
    valid_features = X_val
    valid_labels = y_val
else:
    valid_features = X_test
    valid_labels = y_test


valid_features = valid_features.reshape(-1, 47,1)
valid_labels = valid_labels.reshape(-1)

In [74]:
valid_features_tensor = torch.tensor(valid_features, dtype=torch.float32)
valid_labels_tensor = torch.tensor(valid_labels, dtype=torch.long)


valid_dataset = TensorDataset(valid_features_tensor, valid_labels_tensor)
valid_data_loader = DataLoader(valid_dataset, batch_size=128, shuffle=True)

In [75]:
def validate(model, val_loader, criterion):
    model.eval()  # Set the model to evaluation mode
    val_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():  # No need to track the gradients
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss /= len(val_loader)
    accuracy = 100 * correct / total
    return val_loss, accuracy


In [76]:
# Example usage
val_loss, val_accuracy = validate(model, valid_data_loader, criterion)
print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')


Validation Loss: 0.2285, Validation Accuracy: 94.61%
